# Walmart sales

In [171]:
# Import libraries
import pandas as pd
import numpy as np

import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error 

In [172]:
# Prepare functions for visualizations
def histogram_eda(dataframe, column, target):
    fig = px.histogram(dataframe, x = column, y = target, nbins=10)
    fig.update_layout(showlegend = False, title = f"Distribution of {column}", title_x=0.3)
    fig.update_xaxes(title=None)
    fig.show()
    
def bar_eda(dataframe, column, target):
    fig = px.bar(dataframe, x = column, y = target)
    fig.update_layout(title = f"Representation of {column} regarding {target}", title_x=0.3)
    fig.update_xaxes(title = None)
    fig.show()

In [173]:
# Import the dataset
df = pd.read_csv('Walmart_Store_sales.csv')

# Basic stats
def basic_stats(dataframe):
    print(f"Dataframe shape : {dataframe.shape[0]} rows, {dataframe.shape[1]} columns")
    print()
    print('Count of duplicates rows :', len(dataframe)-len(dataframe.drop_duplicates()))
    print()
    
    print("Display of dataset: ")
    display(dataframe.head())
    
    print("Basics statistics: ")
    display(dataframe.describe(include="all"))

    print("Percentage of missing values: ")
    display(100 * dataframe.isnull().sum() / dataframe.shape[0])

    print('Type for each columns')
    display(dataframe.dtypes)

basic_stats(df)

Dataframe shape : 150 rows, 8 columns

Count of duplicates rows : 0

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

Type for each columns


Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

### Preprocessing the data with pandas

In [174]:
#Drop rows where the target variable is missing
df.dropna(subset=['Weekly_Sales'], inplace=True)
df.shape

(136, 8)

In [175]:
# Drop lines containing invalid values or outliers
# In this project, will be considered as outliers all the numeric features that don't fall within the range +/- 3 std from the average
def remove_outliers(dataframe, columns):
    for col in columns:
        std = dataframe[col].std()
        mean = dataframe[col].mean()
        mask = (dataframe[col] > (mean - 3*std)) & (dataframe[col] < (mean + 3*std))
        dataframe = dataframe[mask]
    return dataframe

outliers = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]
df = remove_outliers(df, outliers)
df.shape

(90, 8)

In [176]:
# Drop lines with missing values in the Date column
df.dropna(subset=['Date'], inplace=True)

#Process the column "Date" and drop the original column
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Week_Number'] = df['Date'].dt.isocalendar().week
df['Weekday'] = df['Date'].dt.dayofweek 

df = df.drop(columns="Date")
df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_Number,Weekday
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011,2,18,7,4
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011,3,25,12,4
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010,5,28,21,4
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011,6,3,22,4
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012,2,3,5,4


Missing values from the data column will need data imputation.

In [177]:
# Check distribution of weekday
df['Weekday'].value_counts()

Weekday
4    80
Name: count, dtype: int64

In [178]:
# Drop weekday column as there is the same day
df = df.drop(columns="Weekday")

### Data visualization

In [179]:
# Check distribution of the year
fig = px.pie(df, names="Year", values="Weekly_Sales", title="Repartition of the sales among the year")
fig.show()

In [180]:
fig = px.pie(df, names="Holiday_Flag", values="Weekly_Sales", title="Repartition of the sales regarding holidays flags")
fig.show()

Missing values will be replaced by the most frequent value (not holiday), but as we would've supposed there is more sales during normal days.

In [181]:
bar_eda(df, "Store", "Weekly_Sales")

Some stores sell more than others. But we can also notice that there is a lack of weekly sales reports for some stores.

In [182]:
numerical_viz = ["Temperature", "Fuel_Price", "CPI", "Unemployment", "Month", "Day", "Week_Number"]

In [183]:
for i in numerical_viz:
    histogram_eda(df, i, "Weekly_Sales")

### Review before machine learning

In [184]:
basic_stats(df)

Dataframe shape : 80 rows, 11 columns

Count of duplicates rows : 0

Display of dataset: 


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_Number
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011,2,18,7
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011,3,25,12
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010,5,28,21
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011,6,3,22
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012,2,3,5


Basics statistics: 


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_Number
count,80.000000,8.000000e+01,71.000000,80.00000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.0
mean,9.575000,1.221522e+06,0.084507,61.12775,3.290700,181.077638,7.301775,2010.887500,6.362500,16.125000,25.4875
std,6.143382,6.799270e+05,0.280126,17.44760,0.491223,38.847021,0.955392,0.826672,3.028321,8.521566,13.188021
min,1.000000,2.689290e+05,0.000000,18.79000,2.548000,126.139200,5.143000,2010.000000,1.000000,1.000000,1.0
25%,4.000000,5.295107e+05,0.000000,45.58750,2.804000,132.610242,6.520750,2010.000000,4.000000,10.000000,15.75
50%,8.000000,1.260826e+06,0.000000,61.45000,3.390500,197.500965,7.345500,2011.000000,6.000000,16.500000,25.0
75%,15.000000,1.817517e+06,0.000000,75.47750,3.689750,214.809008,8.090000,2012.000000,8.250000,23.250000,34.5
max,20.000000,2.771397e+06,1.000000,91.65000,4.170000,226.968844,9.342000,2012.000000,12.000000,31.000000,52.0


Percentage of missing values: 


Store            0.00
Weekly_Sales     0.00
Holiday_Flag    11.25
Temperature      0.00
Fuel_Price       0.00
CPI              0.00
Unemployment     0.00
Year             0.00
Month            0.00
Day              0.00
Week_Number      0.00
dtype: float64

Type for each columns


Store           float64
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
Year              int32
Month             int32
Day               int32
Week_Number      UInt32
dtype: object

In [185]:
# Correlation matrix
corr_matrix = df.corr().round(2)

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

We can clearly see that CPI is an impactful feature, it is logical as the CPI affects the purchasing power.

### Preprocessings with scikit-learn

In [186]:
# Separate target variable Y from features X
target_variable = "Weekly_Sales"
X = df.drop(target_variable, axis=1)
Y = df.loc[:, target_variable]

# Separate train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [187]:
# Assign features to variables
numeric_features = "Temperature", "Fuel_Price", "CPI", "Unemployment", "Year", "Month", "Day", "Week_Number"
categorical_features = "Store", "Holiday_Flag"

# Process numerical features : StandardScaler to scale numerical date
numeric_transformer = StandardScaler()

# Create pipeline for categorical features : OneHotEncoder to encode categorical variables and Simple imputer for missing values
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent"),),  # missing values will be replaced by most frequent value
        ("encoder", OneHotEncoder(drop="first"),),  # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

# Preprocessings on train and test set
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

### Baseline model : linear regression

In [188]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, Y_train)

y_train_pred = linear_regression.predict(X_train)
y_test_pred = linear_regression.predict(X_test)

# Print metrics
lr_R2_train = linear_regression.score(X_train, Y_train)
lr_R2_test = linear_regression.score(X_test, Y_test)


print("R2 score on training set : ", lr_R2_train)
print("R2 score on test set : ", lr_R2_test)
print()
print("Mean Squared Error on training set:", mean_squared_error(Y_train, y_train_pred))
print("Mean Squared Error on test set:", mean_squared_error(Y_test, y_test_pred))

R2 score on training set :  0.9836217813573327
R2 score on test set :  0.9581615416974721

Mean Squared Error on training set: 7434352577.492342
Mean Squared Error on test set: 19419091353.754185


In [189]:
# Display feature importance
coeff_importance = pd.DataFrame({'Feature': preprocessor.get_feature_names_out(), "Coefficients" : np.abs(linear_regression.coef_)})
coeff_importance = coeff_importance.sort_values(by="Coefficients", ascending=True)

# Plot coefficients
fig = px.bar(coeff_importance, x="Coefficients", y="Feature", orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120}, # to avoid cropping of column names
                  height= 600
                 )
fig.show()

As mentionned above, CPI seems to have a clear impact on the target as it is one the main feature. The other important feature is "Store", as we can see that the store number is part of the most impactful feature but this feature does not help as we don't have the same number of reports for every store.

### Regularization models

#### Ridge

In [190]:
# Perform grid search to find the best value for alpha
print("Grid search...")
ridge = Ridge()

# Grid of values to be tested
params = {
    'alpha': np.arange(0,100,0.1)
}

best_ridge = GridSearchCV(ridge, param_grid = params, cv = 10) # cv : the number of folds to be used for CV
best_ridge.fit(X_train, Y_train)
 
print("Best hyperparameters : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)

Grid search...
Best hyperparameters :  {'alpha': 0.0}
Best R2 score :  0.9325658742031049


In [191]:
# Performance assessment of grid search
ridge_R2_train = best_ridge.score(X_train, Y_train)
ridge_R2_test = best_ridge.score(X_test, Y_test)

print("R2 score on training set : ", ridge_R2_train)
print("R2 score on test set : ", ridge_R2_test)

R2 score on training set :  0.9836217813573327
R2 score on test set :  0.9581615416975358


#### Lasso

In [192]:
# Perform grid search to find the best value for alpha
print("Grid search...")
lasso = Lasso()

# Grid of values to be tested
params = {
    'alpha': np.arange(0.1,100,0.1)
}

best_lasso = GridSearchCV(lasso, param_grid = params, cv = 10) # cv : the number of folds to be used for CV
best_lasso.fit(X_train, Y_train)
 
print("Best hyperparameters : ", best_lasso.best_params_)
print("Best R2 score : ", best_lasso.best_score_)

Grid search...


s:\Logiciels\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.691e+11, tolerance: 2.426e+09

s:\Logiciels\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.442e+11, tolerance: 2.666e+09

s:\Logiciels\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.195e+11, tolerance: 2.378e+09

s:\Logiciels\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate

Best hyperparameters :  {'alpha': 45.800000000000004}
Best R2 score :  0.9279693239517945


s:\Logiciels\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.024e+11, tolerance: 2.737e+09

s:\Logiciels\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.270e+11, tolerance: 2.722e+09

s:\Logiciels\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.182e+11, tolerance: 2.800e+09

s:\Logiciels\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate

In [193]:
# Performance assessment of grid search
lasso_R2_train = best_lasso.score(X_train, Y_train)
lasso_R2_test = best_lasso.score(X_test, Y_test)

print("R2 score on training set : ", lasso_R2_train)
print("R2 score on test set : ", lasso_R2_test)

R2 score on training set :  0.9818629139597775
R2 score on test set :  0.974479670901312


#### Comparing models

In [194]:
# Create a dataframe to store all the metrics - easier to compare the models
metrics = {
    'model': ['lr', 'ridge', 'lasso'], 
    'r2_train': [lr_R2_train, ridge_R2_train, lasso_R2_train], 
    'r2_test': [lr_R2_test, ridge_R2_test, lasso_R2_test]}

performance = pd.DataFrame.from_dict(metrics)
performance

,model,r2_train,r2_test
0,lr,0.983622,0.958162
1,ridge,0.983622,0.958162
2,lasso,0.981863,0.974480


In [195]:
data_dict = {
    'Feature': preprocessor.get_feature_names_out(),
    "Linear_Regression" : np.abs(linear_regression.coef_),
    'Best_Ridge': np.abs(best_ridge.best_estimator_.coef_),
    'Best_Lasso': np.abs(best_lasso.best_estimator_.coef_)
            }

coefficients = pd.DataFrame(data = data_dict)
coefficients.head()

,Feature,Linear_Regression,Best_Ridge,Best_Lasso
0,num__Temperature,5.696000e+04,5.696000e+04,53083.993442
1,num__Fuel_Price,8.186158e+04,8.186158e+04,97070.500054
2,num__CPI,1.644945e+06,1.644945e+06,281272.851424
3,num__Unemployment,9.397830e+04,9.397830e+04,60438.296387
4,num__Year,1.032746e+05,1.032746e+05,35418.375126


In [196]:
# Plot coefficients
fig = px.line(coefficients, x="Feature", y=["Linear_Regression", "Best_Ridge", "Best_Lasso"])
fig.update_layout(margin = {'l': 120}, # to avoid cropping of column names
                  height= 600
                 )
fig.show()

In [197]:
X.columns

Index(['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment', 'Year', 'Month', 'Day', 'Week_Number'],
      dtype='object')

In [198]:
df

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_Number
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011,2,18,7
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011,3,25,12
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010,5,28,21
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011,6,3,22
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...
139,7.0,532739.77,0.0,50.60,3.804,197.588605,8.090,2012,5,25,21
143,3.0,396968.80,0.0,78.53,2.705,214.495838,7.343,2010,6,4,22
144,3.0,424513.08,0.0,73.44,3.594,226.968844,6.034,2012,10,19,42
145,14.0,2248645.59,0.0,72.62,2.780,182.442420,8.899,2010,6,18,24


In [211]:
# Predict a result
X_new = pd.DataFrame(data = [[3,1,60.1,2.759, 210.52, 7.254, 2011, 4, 14, 15]], 
                     columns = ['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI',
                                'Unemployment', 'Year', 'Month', 'Day', 'Week_Number'])
X_new = preprocessor.transform(X_new)

result = best_lasso.predict(X_new)
print("The prediction is :", int(result), "dollars")

The prediction is : 284645 dollars


## Conclusion
Linear regression and ridge models have the same resultats and a better R2 score on the train set. 
Lasso model has a lower score on the train set but we can see that the R2 score is higher on the test set. The difference between the two scores is lower than the simple linear regression model.